In [5]:
from model.data import *
from model.model import *
d = Data()

In [2]:
import os

config = {}
config["weights_file"] = os.getcwd() + '/model/weight'
config["patch_size"] = (64, 64, 64)  # switch to None to train on the whole image
config["patch_gap"] = 16
config["batch_size"] = 10
config["kfold"] = 5

config["input_shape"] = (1, 64, 64, 64)
config["depth"] = 4 # depth of layers for V/Unet
config["n_base_filters"] = 32
config["pool_size"] = (2, 2, 2)  # pool size for the max pooling operations
config["deconvolution"] = True  # if False, will use upsampling instead of deconvolution

config["patience"] = 10  # learning rate will be reduced after this many epochs if the validation loss is not improving
config["early_stop"] = 10  # training will be stopped after this many epochs without the validation loss improving
config["initial_learning_rate"] = 0.00001
config["learning_rate_drop"] = 0.5  # factor by which the learning rate will be reduced
config["n_epochs"] = 1

model = unet_model_3d(input_shape=config["input_shape"],
                              pool_size=config["pool_size"],
                              initial_learning_rate=config["initial_learning_rate"],
                              deconvolution=config["deconvolution"],
                              depth=config["depth"],
                              n_base_filters=config["n_base_filters"])

model.summary()
counter = 1
for layer in model.layers:
    print(counter, "&", layer.name, "&", layer.output_shape)
    counter += 1


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 64, 64, 64 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 32, 64, 64, 6 896         input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 64, 64, 6 0           conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 64, 64, 64, 6 55360       activation_1[0][0]               
__________________________________________________________________________________________________
activation

In [6]:
# 2 & conv3d_1 & (None, 32, 64, 64, 64)
# 3 & activation_1 & (None, 32, 64, 64, 64)
# 4 & conv3d_2 & (None, 32, 64, 64, 64)
# 5 & activation_2 & (None, 32, 64, 64, 64)
# 6 & max_pooling3d_1 & (None, 32, 32, 32, 32)
counter = 1
filt = 32
lay = 64
# while (filt < 512):
# #     print("Contract " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
#     print("Contract " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
#     lay = int(lay / 2)
#     print("Contract " + str(counter), "&", "Max pooling & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
#     filt *= 2
#     counter += 1   

while (filt < 512):
#     print("Contract " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    print("Contract " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
#     lay = int(lay / 2)
#     print("Contract " + str(counter), "&", "Max pooling & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    filt *= 2
    print("Contract " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    lay = int(lay / 2)
    print("Contract " + str(counter), "&", "Max pooling & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
#     filt *= 2
    counter += 1    

    
#     2 & Conv3D + ReLU & (64, 64, 64, 64) \\
#     2 & Max pooling & (64, 32, 32, 32) \\

Contract 1 & Conv3D + ReLU & (32, 64, 64, 64) \\
Contract 1 & Conv3D + ReLU & (64, 64, 64, 64) \\
Contract 1 & Max pooling & (64, 32, 32, 32) \\
Contract 2 & Conv3D + ReLU & (64, 32, 32, 32) \\
Contract 2 & Conv3D + ReLU & (128, 32, 32, 32) \\
Contract 2 & Max pooling & (128, 16, 16, 16) \\
Contract 3 & Conv3D + ReLU & (128, 16, 16, 16) \\
Contract 3 & Conv3D + ReLU & (256, 16, 16, 16) \\
Contract 3 & Max pooling & (256, 8, 8, 8) \\
Contract 4 & Conv3D + ReLU & (256, 8, 8, 8) \\
Contract 4 & Conv3D + ReLU & (512, 8, 8, 8) \\
Contract 4 & Max pooling & (512, 4, 4, 4) \\


In [9]:
# 21 & conv3d_transpose_1 & (None, 512, 16, 16, 16)
# 22 & concatenate_1 & (None, 768, 16, 16, 16)
# 23 & conv3d_9 & (None, 256, 16, 16, 16)
# 24 & activation_9 & (None, 256, 16, 16, 16)
# 25 & conv3d_10 & (None, 256, 16, 16, 16)
# 26 & activation_10 & (None, 256, 16, 16, 16)

lay = 16
counter = 3
filt = 512
while (counter >= 1):
    print("Expand " + str(counter), "&", "Deconvolution3D & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    print("Expand " + str(counter), "&", "Concatenation & (" + str(int(filt * 1.5)) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    filt = int(filt/2)
    print("Expand " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    lay *= 2
    counter -= 1

# lay = 8
# counter = 4
# filt = 256
# while (counter >= 1):
#     print("Expand " + str(counter), "&", "Deconvolution3D & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
#     print("Expand " + str(counter), "&", "Concatenation & (" + str(filt * 2) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
#     print("Expand " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
#     filt = int(filt/2)
#     lay *= 2
#     counter -= 1


Expand 3 & Deconvolution3D & (512, 16, 16, 16) \\
Expand 3 & Concatenation & (768, 16, 16, 16) \\
Expand 3 & Conv3D + ReLU & (256, 16, 16, 16) \\
Expand 2 & Deconvolution3D & (256, 32, 32, 32) \\
Expand 2 & Concatenation & (384, 32, 32, 32) \\
Expand 2 & Conv3D + ReLU & (128, 32, 32, 32) \\
Expand 1 & Deconvolution3D & (128, 64, 64, 64) \\
Expand 1 & Concatenation & (192, 64, 64, 64) \\
Expand 1 & Conv3D + ReLU & (64, 64, 64, 64) \\


In [1]:
file = [0,0.6774007777101638,0.32259922231444055,0.6014426891388959,0.39855731060899396,
1,0.8791109366527453,0.12088906335315987,0.6255426337696927,0.3744573660986723,
2,0.9215844093029602,0.07841559069159167,0.6227154042962295,0.3772845954458783,
3,0.9408884294507033,0.05911157054799623,0.6270633373407892,0.37293666257446517,
4,0.9527984541957688,0.047201545801067696,0.6272190573076512,0.37278094252454486,
]

print(len(file))
for i in range(int(len(file)/5)):
    print(str(file[i * 5] + 1) + ' & ', end="")
    for j in range(1, 5):
        if j != 4:
            print("{0:.4f}".format(file[i*5 + j]) + ' & ', end="")
        else:
            print("{0:.4f}".format(file[i*5 + j]) + ' \\\\')
                

25
1 & 0.6774 & 0.3226 & 0.6014 & 0.3986 \\
2 & 0.8791 & 0.1209 & 0.6255 & 0.3745 \\
3 & 0.9216 & 0.0784 & 0.6227 & 0.3773 \\
4 & 0.9409 & 0.0591 & 0.6271 & 0.3729 \\
5 & 0.9528 & 0.0472 & 0.6272 & 0.3728 \\


In [ ]:
# 1 & 0.6996 & 0.3004 & 0.6416 & 0.3584 \\
# 1 & 0.6918 & 0.3082 & 0.6207 & 0.3793 \\
# 2 & 0.8909 & 0.1091 & 0.6614 & 0.3386 \\
# 2 & 0.8930 & 0.1070 & 0.5215 & 0.4785 \\


% binary: 
1 & 0.5122 & 0.0245 & 0.5665 & 0.0353 \\
2 & 0.8538 & 0.0041 & 0.5879 & 0.0518 \\
3 & 0.9160 & 0.0025 & 0.5985 & 0.0580 \\
4 & 0.9450 & 0.0017 & 0.6130 & 0.0550 \\
5 & 0.9634 & 0.0012 & 0.6266 & 0.0626 \\

% normal:
1 & 0.6566 & 0.3434 & 0.6013 & 0.3987 \\
2 & 0.8843 & 0.1157 & 0.5048 & 0.4952 \\
3 & 0.9240 & 0.0760 & 0.6145 & 0.3855 \\
4 & 0.9425 & 0.0575 & 0.5981 & 0.4019 \\
5 & 0.9544 & 0.0456 & 0.6021 & 0.3979 \\

% all:
1 & 0.6220 & 0.3780 & 0.6039 & 0.3961 \\
2 & 0.6385 & 0.3615 & 0.6039 & 0.3961 \\
3 & 0.6385 & 0.3615 & 0.6039 & 0.3961 \\
4 & 0.6385 & 0.3615 & 0.6039 & 0.3961 \\
5 & 0.6385 & 0.3615 & 0.6039 & 0.3961 \\

